In [0]:
import pandas as pd
import numpy as np
import sys
import re
import gc
import csv

from google.colab import drive

!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

stopwords = [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
"yourselves"]

print(len(stopwords))

drive.mount('/content/drive')
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 23.4MB/s 
318
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
troll_file_list = drive.ListFile({'q': "'1IMT5wkteYw7oVhN5exjoHmFtGaWSVu28' in parents and trashed=false"}).GetList()

for troll_file1 in troll_file_list:
  
  print('title: %s, id: %s' % (troll_file1['title'], troll_file1['id']))

title: IRAhandle_tweets_8.csv, id: 1tncgM0Qk8tVwnlDpcS6ExcKLgXSQFivr
title: IRAhandle_tweets_6.csv, id: 1Zj7y_-3A-7Fos-zGCHEKgPBoQvjmysCS
title: IRAhandle_tweets_7.csv, id: 1jI5nQyznspfgk6x1RqJeYs184R8rDAH3
title: IRAhandle_tweets_9.csv, id: 14fs5-wJUPRJ5_n_KNWOFkXOF_JgQBcvn
title: IRAhandle_tweets_5.csv, id: 1zqwycOdInUlWqs6n1cg1O9ahs-LdfDH-
title: IRAhandle_tweets_3.csv, id: 1Rk7XNjnBzeCPi2o-TV1pD8qbMFOxrnXH
title: IRAhandle_tweets_4.csv, id: 1QSArmXCXdkfWHlcTVqPwnzdAjjkOenF7
title: IRAhandle_tweets_2.csv, id: 1bNSGGniOObCfzTZ085FyFVjqNpC5cRnD
title: IRAhandle_tweets_1.csv, id: 1TEkomSxdkUuhkpWTvB1YNZFsvEVP_Pwk


In [0]:
trolldf = pd.DataFrame()

for troll_file in troll_file_list :
  
  downloaded = drive.CreateFile({'id' : troll_file['id']})
  
  downloaded.GetContentFile(troll_file['title'])
  
  trolldf = trolldf.append( pd.read_csv(troll_file['title'], usecols = ['author','content']), ignore_index = True, sort = False)
           
trolldf.head()

,author,content
0,STAYWOKE88,#Illinois Dairy #Queen Franchisee Loses Store ...
1,STAYWOKE88,#NoEthicsNoHearing https://t.co/q8suwAO2sB
2,STAYWOKE88,Progressive narratives about what’s driving ma...
3,STAYWOKE88,Finally found his twin. Mitch McConnell https:...
4,STAYWOKE88,"#DonnieDipshit: ""Confirmations r going great""...."


In [0]:
troll_zeros = [0] * len(stopwords)

troll_stop_word_dict = dict(zip(stopwords, troll_zeros))

troll_per_profile_tweets = trolldf.groupby('author')['content']

troll_per_profile_tweets.head()

0          #Illinois Dairy #Queen Franchisee Loses Store ...
1                 #NoEthicsNoHearing https://t.co/q8suwAO2sB
2          Progressive narratives about what’s driving ma...
3          Finally found his twin. Mitch McConnell https:...
4          #DonnieDipshit: "Confirmations r going great"....
5509       как погибала 131-я Майкопская бригада,1 января...
5510       Ничего особенного. Просто Пется нахерачился и ...
5511       The good, the bad &amp; the ugly: Remembering ...
5512       You better understand it, Zionism and the Zion...
5513       Около тысячи украинцев приняли участие в факел...
6841       Болельщики минского «Динамо» освистали Стаса Д...
6842       Draft changes to &amp;quot;parasite&amp;quot; ...
6843       Основатель знаменитого Playboy умер от останов...
6844       Узбекские Chevrolet будут продаваться в Белару...
6845       velcom запустил в Беларуси «индустриальный инт...
6995          Ужасы школьной практики http://t.co/rFM1wde5m6
6996       Ученые из Сиб

In [0]:
handle1 = open('/tmp/train.csv', 'w')
handle2 = open('/tmp/test.csv', 'w')

writer1 = csv.writer(handle1)
writer2 = csv.writer(handle2)

temp = ['author']

for s in stopwords :
  temp.append(s)


temp.append('troll')

writer1.writerow(temp)
writer2.writerow(temp)



1924

In [0]:
! cat /tmp/train.csv
! cat /tmp/test.csv

In [0]:
troll_idx = 0

troll_x = []

cnt = 2130


for troll_key, troll_grp in troll_per_profile_tweets:
    
    troll_idx = troll_idx + 1
  
    print('idx : ', troll_idx, ' author : ', troll_key)
    
    troll_x.append(troll_key)
    
    
    troll_length = 0
    
    for troll_s in troll_grp:
      
        troll_y = str(troll_s).split(' ')

        troll_length = troll_length + len(troll_y)

        for troll_word in [troll_w.lower() for troll_w in troll_y  if len(troll_w) < 15]: 

            if troll_word in troll_stop_word_dict:

                troll_stop_word_dict[troll_word] = troll_stop_word_dict[troll_word] + 1
                
        troll_y.clear()
        
        
      
    for troll_w in list(troll_stop_word_dict.values()) :
      
      troll_x.append(troll_w / troll_length)
      
    troll_x.append(1)
    
    if troll_idx < 2130 :
      writer1.writerow(troll_x)
    else :
      writer2.writerow(troll_x)
    
    
    
    troll_x.clear()
    

    
    for troll_k in troll_stop_word_dict.keys():
        
        troll_stop_word_dict[troll_k] = 0
        

idx :  1  author :  10_GOP
idx :  2  author :  1488REASONS
idx :  3  author :  1D_NICOLE_
idx :  4  author :  1ERIK_LEE
idx :  5  author :  1LORENAFAVA1
idx :  6  author :  2NDHALFONION
idx :  7  author :  459JISALGE
idx :  8  author :  4EVER1937
idx :  9  author :  4EVER_SUSAN
idx :  10  author :  4MYSQUAD
idx :  11  author :  5EMEN5EMENICH
idx :  12  author :  5L5_5L5
idx :  13  author :  666STEVEROGERS
idx :  14  author :  6DRUZ
idx :  15  author :  71BILALJAMIL1
idx :  16  author :  AAASSSSSHHH
idx :  17  author :  AAATEST1234
idx :  18  author :  AAAVADAKEDAVRAA
idx :  19  author :  AAGN1AA
idx :  20  author :  AANTIRACIST
idx :  21  author :  AARONALLENALL
idx :  22  author :  AARON_M1TCHELL
idx :  23  author :  ABALAKOVAYLIA
idx :  24  author :  ABBASSHUKRI
idx :  25  author :  ABBYLOPTRT
idx :  26  author :  ABELLABAB
idx :  27  author :  ABIGAILCALLME
idx :  28  author :  ABIGAILSSILK
idx :  29  author :  ABIISSROSB
idx :  30  author :  ABISADMASST
idx :  31  author :  ABMERRL

TROLL - NORMAL-BORDER

In [0]:
normal_file_list = drive.ListFile({'q': "'1sFcNq_jqE8-vdmC1TlXrh2plS8qy84PZ' in parents and trashed=false"}).GetList()

for normal_file1 in normal_file_list:
  
  print('title: %s, id: %s' % (normal_file1['title'], normal_file1['id']))

title: nov29.csv, id: 12NSIxIbgiQd8N3i49lcK4DeBpaV6hCWc
title: nov28.csv, id: 1huayYTurbpgpsVI99IoKVyUBkdKGhz_N
title: nov27.csv, id: 1AAAUuIdyq1TjZgO54Njl3s4E4hDzeTAK
title: nov30.csv, id: 1cmPemTfEF61B2TWpwO1f8h9WQxIdxKf8


In [0]:
normaldf = pd.DataFrame()

for normal_file in normal_file_list:
    
    downloaded = drive.CreateFile({'id': normal_file['id']})
    
    downloaded.GetContentFile(normal_file['title'])
    
    normaldf = normaldf.append( pd.read_csv(normal_file['title'], usecols = ['author_id','content']), ignore_index = True, sort = False)
    
    
    
normaldf.columns = trolldf.columns
normaldf.head()


,author,content
0,225551004,Download this epic mid-tempo jam #AiVaan by @T...
1,184621766,🌚✅FROOT SHIRT ORDERED✅🌚
2,743166519157227520,RT @WalshFreedom: To the stupid Left in this c...
3,53706363,soundtrack ตอนนี้ คงเป็นเพลงนี้ https://t.co/d...
4,2418453842,RT @toyama1224: デビュー以来２１年間、熱愛報道は１回のみしかもその相手と結婚...


In [0]:
normal_zeros = [0] * len(stopwords)

normal_stop_word_dict = dict(zip(stopwords, normal_zeros))

normal_per_profile_tweets = normaldf.groupby('author')['content']

normal_per_profile_tweets.head()

0           Download this epic mid-tempo jam #AiVaan by @T...
1                                     🌚✅FROOT SHIRT ORDERED✅🌚
2           RT @WalshFreedom: To the stupid Left in this c...
3           soundtrack ตอนนี้ คงเป็นเพลงนี้ https://t.co/d...
4           RT @toyama1224: デビュー以来２１年間、熱愛報道は１回のみしかもその相手と結婚...
5           RT @selbsie: @menggz24 @Fan_Reel wala bang reg...
6           RT @HuffingtonPost: Baby goat can only truly r...
7           Not everybody will find love\nNot everybody wi...
8           7 Things You need Know About Your Smartphone (...
9           These 15 facts about the Bermuda Triangle will...
10          RT @creamsoda1013: 161128 #지민 #JIMIN 왜내마믈흔드는겅데...
11          RT @TODAYonline: JUST IN: South Korean Preside...
12          watch n share "Maire Ahaji The Video"  mad vid...
13          RT @jPWus26XvKN9bdr: يقولون ان عودته مستحيله\n...
14          Blast from the past. Where is Spencer Prior no...
15          [To Read] The Many Advantages Of Being On Tour...
16      

In [0]:
normal_idx = 0

cnt = 1500000

normal_x = []



for normal_key, normal_grp in normal_per_profile_tweets:
    
    normal_idx = normal_idx + 1
  
    # print('idx : ', normal_idx, ' author : ', normal_key)
    
    normal_x.append(normal_key)
    
    
    normal_length = 0
    
    for normal_s in normal_grp:
      
        normal_y = str(normal_s).split(' ')

        normal_length = normal_length + len(normal_y)

        for normal_word in [normal_w.lower() for normal_w in normal_y  if len(normal_w) < 15]: 

            if normal_word in normal_stop_word_dict:

                normal_stop_word_dict[normal_word] = normal_stop_word_dict[normal_word] + 1
                
        normal_y.clear()
        
        
      
    for normal_w in list(normal_stop_word_dict.values()) :
      
      normal_x.append(normal_w / normal_length)
      
    normal_x.append(0)
    
    if normal_idx < cnt :
      if normal_idx < 1125000 :
        writer1.writerow(normal_x)
      else :
        writer2.writerow(normal_x)
    else :
      
      #normal_writer_test.writerow(normal_x)
      break
    
    
    normal_x.clear()
    

    
    for normal_k in normal_stop_word_dict.keys():
        
        normal_stop_word_dict[normal_k] = 0
        
        

In [0]:
! wc -l /tmp/train.csv
! wc -l /tmp/test.csv

1127127 /tmp/train.csv
375718 /tmp/test.csv


TRAIN - MODEL

In [0]:
stopwords.insert(0, 'author')

X_train = pd.read_csv('/tmp/train.csv', usecols = stopwords)

X_test = pd.read_csv('/tmp/test.csv', usecols = stopwords)




y_train = pd.read_csv('/tmp/train.csv', usecols = ['author', 'troll'])

y_test = pd.read_csv('/tmp/test.csv', usecols = ['author', 'troll'])



X_train.set_index('author', drop=True, append=False, inplace=True, verify_integrity=False)

X_test.set_index('author', drop=True, append=False, inplace=True, verify_integrity=False)

y_train.set_index('author', drop=True, append=False, inplace=True, verify_integrity=False)

y_test.set_index('author', drop=True, append=False, inplace=True, verify_integrity=False)



/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:

Log_clf = LogisticRegression(solver = 'lbfgs').fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:

predicted = Log_clf.predict(X_test)
print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))

Accuracy : {:.2f} 0.9981262243420492
Precision : {:.2f} 0.0
Recall : {:.2f} 0.0
f1-score : {:.2f} 0.0


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:

Dt_clf = DecisionTreeClassifier().fit(X_train, y_train)

In [0]:
predicted = Dt_clf.predict(X_test)
print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))

Accuracy : {:.2f} 0.9988634911847373
Precision : {:.2f} 0.7194928684627575
Recall : {:.2f} 0.6448863636363636
f1-score : {:.2f} 0.6801498127340824


In [0]:

GBDT_clf = GradientBoostingClassifier().fit(X_train, y_train)

In [0]:
predicted = GBDT_clf.predict(X_test)
#len(predicted)
len(y_test)

375719

In [0]:

print('Accuracy : {:.2f}', format(accuracy_score(y_test, predicted)))
print('Precision : {:.2f}', format(precision_score(y_test, predicted)))
print('Recall : {:.2f}', format(recall_score(y_test, predicted)))
print('f1-score : {:.2f}', format(f1_score(y_test, predicted)))

Accuracy : {:.2f} 0.9978867185316687
Precision : {:.2f} 0.41134751773049644
Recall : {:.2f} 0.24200278164116829
f1-score : {:.2f} 0.3047285464098074
